<h1>Ensuring we take correct nodes</h1>
<p>From the files we have. Ensure their length is meaningful. As we will see, we shall proceed with the nodes in reformated.tsv</p>

In [11]:
import os

# Specify the file path
files = os.listdir("../data/")

for a_ile in files:
    file_path = f"../data/{a_ile}"

    # Initialize a line counter
    line_count = 0

    # Open the file in read mode
    with open(file_path, "r") as file:
        # Iterate through each line in the file
        for line in file:
            # Increment the line counter for each line
            line_count += 1

    # Print the total number of lines in the file
    print(f"Number of lines in '{a_ile}': {line_count}")


Number of lines in 'CEN.tsv': 92051051
Number of lines in 'exosome.csv': 14695476
Number of lines in 'raw.tsv': 13989436
Number of lines in 'reformatted.tsv': 13989436
Number of lines in 'first_cluster_edges.tsv': 98753
Number of lines in 'first_cluster_nodes.tsv': 2004


<p>We ended up using reformatted clustering result, and bellow we ensure nodes are unique</p>

In [12]:
from tqdm import tqdm
import csv

def open_nodes(file='../data/reformatted.tsv'):
    data = []
    with open(file, 'r', newline='') as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in tqdm(csv_reader, total = 13989436):
            data.append(int(row[0].split()[0]))

    return data

data = open_nodes()
data.sort()

for i in range(len(data)-1):
    assert data[i] != data[i+1]
    

100%|██████████| 13989436/13989436 [00:15<00:00, 892517.56it/s]


In [13]:
print(data[0:10])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [14]:
import os
import psycopg2
import json

os.environ["PGDATABASE"] = "ernieplus"

conn = psycopg2.connect("")
cur = conn.cursor()

#number stats and missing stats
stats = {}


def return_single_record(PMID, cur):
    sql_query = f"select * from hm31.pubmed_all_xmls_new an where pmid = {PMID}"
    cur.execute(sql_query)
    result = cur.fetchall()
    
    return result
    

missed = 0
missing_numbers = []

for _, pmid in tqdm(enumerate(data), total = 13989436):
    result = return_single_record(pmid,cur)
    
    if len(result) > 1:
        stats[pmid] = 'multi'
    
    elif len(result) == 0:
        missed += 1
        missing_numbers.append(pmid)
        

stats['missed'] = missed

print(missing_numbers)
        
        
with open('CEN_stats.json', "w") as json_file:
    json.dump(stats, json_file)

    
 

    
 

100%|██████████| 13989436/13989436 [11:52<00:00, 19640.79it/s]
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
print(missed)

316108


In [17]:
print(len(data))

13989436


In [19]:
print(missing_numbers[0:10])
for i in missing_numbers[0:10]:
    print(i in data)

[0, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062]
True
True
True
True
True
True
True
True
True
True


In [ ]:
import os

# Set the PGDATABASE environment variable
os.environ["PGDATABASE"] = "ernieplus"

import psycopg2

conn = psycopg2.connect("")
print("Connected to Postgres successfully")
# conn.close()

# Create a cursor object to interact with the database
cur = conn.cursor()

# Define your SQL query
sql_query = "select * from hm31.aligned_musicians am limit 10"

# Execute the SQL query
cur.execute(sql_query)

# Fetch and print the result (you can modify this part based on your query)
result = cur.fetchall()
for row in result:
    print(row)

<h1> Sample nodes </h1>
<p> Sample random nodes for studying distance </p>

In [ ]:
import json
import random
import os
import psycopg2
random.seed(4311)


with open('../step_1_sample_study/first_cluster_pmid.json', 'r') as file:
    # Load the JSON data into a dictionary
    meta_data_dict = json.load(file)
    
first_pmids = list(meta_data_dict.keys())

In [ ]:
os.environ["PGDATABASE"] = "ernieplus"

conn = psycopg2.connect("")
cur = conn.cursor()

random_selection = random.sample(data, 3000)

final_selected = []

for index, pmid in tqdm(enumerate(random_selection)):
    result = return_single_record(pmid,cur)
    #print(result, len(result))
    if len(result) > 0 and (not pmid in first_pmids):
        final_selected.append(pmid)
    
    if len(final_selected) == 2004:
        break

In [ ]:
salvo = {'random': final_selected, 'first_cluster': [int(f) for f in first_pmids] }
# print(salvo)

file_path = 'pmids.json'

# Open the file for writing
with open(file_path, 'w') as json_file:
    # Write the 'salvo' dictionary to the JSON file
    json.dump(salvo, json_file, indent=4)
    
!ls

In [ ]:
print(len(salvo['random']), len(salvo['first_cluster']))